In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from bayes_opt import BayesianOptimization
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import BaseCrossValidator
from sklearn.linear_model import Ridge
import time
from sklearn.metrics import make_scorer
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
df = pd.read_excel('Data diolah darin.xlsx')
df

,Backhoe,HP(watt),Norm_MAP,RPM,Age,Load_Factor,Engine_Tier,TEMP[C],Fuel[g/s],NOx[g/s],...,NOx[g/hr],HC[g/hr],CO[g/hr],CO2[g/hr],PM[mg/hr],Nox (g/kl),HC (g/kl),CO (g/kl),CO2 (g/kl),PM (g/kl)
0,1,65621.6,0.000000,833,12,0.21,2,22,0.430000,0.024090,...,86.724000,25.848000,13.176000,4780.656000,14.4,3730.117500,1111.757728,566.717728,2.056225e+05,0.619362
1,1,65621.6,0.012346,800,12,0.21,2,22,0.620000,0.030320,...,109.152000,25.668000,16.164000,7008.264000,21.6,4694.776362,1104.015681,695.235681,3.014350e+05,0.929047
2,1,65621.6,0.012346,826,12,0.21,2,22,0.510000,0.030730,...,110.628000,10.980000,10.584000,5722.920000,21.6,4758.261138,472.264772,455.232272,2.461506e+05,0.929047
3,1,65621.6,0.012346,831,12,0.21,2,22,0.390522,0.023693,...,85.293664,20.747853,4.825082,4382.553038,21.6,3668.596780,892.393432,207.533355,1.884996e+05,0.929047
4,1,65621.6,0.012346,834,12,0.21,2,22,0.413024,0.026107,...,93.985262,6.979907,9.880556,4673.503069,21.6,4042.434273,300.215326,424.976193,2.010137e+05,0.929047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37523,8,72332.9,0.975904,1676,36,0.21,2,41,1.590000,0.058830,...,211.788000,19.908000,28.152000,17957.592000,1800.0,445343.100000,41862.100000,59197.400000,3.776083e+07,3785.000000
37524,8,72332.9,0.975904,1683,36,0.21,2,42,1.320000,0.059500,...,214.200000,19.800000,30.780000,14905.044000,1656.0,542545.340900,50151.250000,77962.397730,3.775286e+07,4194.468182
37525,8,72332.9,0.975904,1846,36,0.21,2,42,1.530000,0.059640,...,214.704000,21.744000,33.840000,17349.048000,1872.0,469179.694100,47515.850980,73948.509800,3.791183e+07,4090.768627
37526,8,72332.9,0.987952,1876,36,0.21,2,42,1.760000,0.056200,...,202.320000,22.356000,34.740000,19948.572000,1980.0,384340.943200,42468.990340,65994.485800,3.789568e+07,3761.343750


In [ ]:
df.replace('#DIV/0!', np.nan, inplace=True)
df.dropna(inplace=True)

C:\Users\TI\AppData\Local\Temp\ipykernel_5960\2881342136.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('#DIV/0!', np.nan, inplace=True)


In [ ]:
X = df[['Backhoe', 'HP(watt)', 'Norm_MAP', 'RPM', 'Age', 'Load_Factor', 'Engine_Tier']]
y = df['NOx[g/s]']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

y_train = y_train.ravel()
y_test = y_test.ravel()

C:\Users\TI\AppData\Local\Temp\ipykernel_5960\1236518287.py:3: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_train = y_train.ravel()
C:\Users\TI\AppData\Local\Temp\ipykernel_5960\1236518287.py:4: FutureWarning: Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy()` for conversion to a numpy array instead.
  y_test = y_test.ravel()


In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(26268, 7)
(26268,)
(11259, 7)
(11259,)


GBR MODEL

In [ ]:
gbr = GradientBoostingRegressor(loss='squared_error')

In [ ]:
gbr.fit(X_train,y_train)
y_pred = gbr.predict(X_test)

In [ ]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

def mape(y_true, y_pred):
    return mean_absolute_percentage_error(y_true, y_pred)

rmse_score = rmse(y_test, y_pred)
mape_score = mape(y_test, y_pred)

print(f'RMSE: {rmse_score:.2f}')
print(f"MAPE: {mape_score:.2f}%")

RMSE: 0.01
MAPE: 0.16%


GRID

In [ ]:
params = {
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'n_estimators': np.arange(1, 51, 5),
    'subsample': np.arange(0.5, 1.0, 0.1),
    'min_samples_split': range(2, 11),
    'min_samples_leaf': range(1, 11),
    'max_depth': range(1, 26)
}

rmse_scorer = make_scorer(mean_squared_error, squared=False)
mape_scorer = make_scorer(mean_absolute_percentage_error)

In [ ]:
grid = GridSearchCV(
    estimator=gbr,
    param_grid=params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    cv=2,
    verbose=3,
    n_jobs=-1
)

start_time = time.time()
gbr_result = grid.fit(X_train,y_train)

print("Best: %f using %s" % (grid.best_score_, grid.best_params_))
print("Best RMSE:", grid.best_score_)
print("Best MAPE:", grid.cv_results_['mean_test_MAPE'][grid.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Fitting 2 folds for each of 1012500 candidates, totalling 2025000 fits
Best: 0.018329 using {'learning_rate': 0.1, 'max_depth': 1, 'min_samples_leaf': 7, 'min_samples_split': 10, 'n_estimators': 1, 'subsample': 0.5}
Best RMSE: 0.01832934237983276
Best MAPE: 0.8348894835467364
Execution time: 52924.7911670208 s


In [ ]:
best_grid = grid.best_estimator_
y_pred_grid = best_grid.predict(X_test)
rmse_grid = np.sqrt(mean_squared_error(y_test, y_pred_grid))
mape_grid = mean_absolute_percentage_error(y_test, y_pred_grid)
print(f"RMSE: {rmse_grid:.2f}")
print(f"MAPE: {mape_grid:.2f}%")

RMSE: 0.02
MAPE: 0.82%


In [ ]:
# export to excel
pd.DataFrame(grid.cv_results_).to_excel('GBR_GS_unscal.xlsx', index=False)

random search

In [ ]:
rs = RandomizedSearchCV(
    gbr,
    params,
    scoring={'RMSE': rmse_scorer, 'MAPE': mape_scorer},
    refit='RMSE',
    n_iter=100,
    cv=5,
    verbose=0,
    n_jobs=-1
)

start_time = time.time()
rs_result = rs.fit(X_train,y_train)

print("Best: %f using %s" % (rs.best_score_, rs.best_params_))
print("Best RMSE:", rs.best_score_)
print("Best MAPE:", rs.cv_results_['mean_test_MAPE'][rs.best_index_])
print("Execution time: " + str((time.time() - start_time)) + ' s')

Best: 0.017384 using {'subsample': 0.8999999999999999, 'n_estimators': 1, 'min_samples_split': 3, 'min_samples_leaf': 6, 'max_depth': 1, 'learning_rate': 0.2}
Best RMSE: 0.017383819834909508
Best MAPE: 0.7862054954861285
Execution time: 21.32726263999939 s


In [ ]:
best_rs = rs.best_estimator_
y_pred_rs = best_rs.predict(X_test)
rmse_rs = np.sqrt(mean_squared_error(y_test, y_pred_rs))
mape_rs = mean_absolute_percentage_error(y_test, y_pred_rs)
print(f"RMSE: {rmse_rs:.2f}")
print(f"MAPE: {mape_rs:.2f}%")

RMSE: 0.02
MAPE: 0.77%


In [ ]:
rs.cv_results_

{'mean_fit_time': array([0.67380352, 0.04019594, 0.79421158, 0.51878066, 0.56596045,
        1.39581637, 1.31722722, 0.82407031, 1.29222951, 0.6738708 ,
        0.54533253, 0.36929312, 0.73042088, 0.02041855, 0.08344855,
        0.39851246, 0.02821379, 0.67773724, 1.37306004, 0.03389688,
        0.40814853, 0.56390085, 0.30889177, 0.85314631, 0.02446027,
        0.34352269, 0.94828844, 0.29846745, 1.83856544, 1.30030875,
        0.92571363, 0.30197001, 0.01767812, 1.03746352, 0.77986016,
        0.51899805, 0.0302165 , 0.54937153, 0.44535413, 0.78993025,
        0.3580195 , 0.02696824, 0.58066001, 0.11925845, 1.80623107,
        0.71054101, 0.6099844 , 0.2975368 , 0.90491152, 0.65789566,
        0.04836407, 0.01208687, 0.47210355, 0.31976881, 1.45742149,
        0.19017272, 0.21122956, 0.23666782, 1.17522879, 1.55510688,
        1.37421455, 0.15706172, 0.36129975, 0.88668919, 0.66185083,
        0.29195833, 0.68648782, 1.29325581, 0.52809143, 0.73160543,
        0.57034168, 0.5118413 ,

In [ ]:
pd.DataFrame(rs.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_subsample,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_depth,param_learning_rate,...,std_test_RMSE,rank_test_RMSE,split0_test_MAPE,split1_test_MAPE,split2_test_MAPE,split3_test_MAPE,split4_test_MAPE,mean_test_MAPE,std_test_MAPE,rank_test_MAPE
0,0.673804,0.044147,0.014619,0.002648,0.6,26,8,2,13,0.6,...,0.000162,27,0.159309,0.159517,0.160987,0.163833,0.165377,0.161805,0.002409,38
1,0.040196,0.005841,0.008758,0.004110,0.7,1,5,4,16,0.3,...,0.000270,4,0.650710,0.643216,0.615176,0.655328,0.626108,0.638107,0.015175,4
2,0.794212,0.048159,0.011706,0.006308,0.8,41,7,8,7,0.8,...,0.000089,66,0.150571,0.150455,0.149928,0.156483,0.150402,0.151568,0.002467,77
3,0.518781,0.024125,0.008076,0.005558,0.6,26,6,7,10,0.6,...,0.000087,65,0.148362,0.149838,0.146907,0.154764,0.150891,0.150152,0.002670,84
4,0.565960,0.039753,0.009714,0.002873,0.6,21,6,9,14,0.8,...,0.000131,37,0.159172,0.161040,0.159246,0.164501,0.160354,0.160863,0.001949,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1.149947,0.037941,0.009449,0.003979,0.9,21,6,9,24,0.7,...,0.000115,48,0.154042,0.157546,0.155153,0.161350,0.155373,0.156693,0.002591,59
96,0.157885,0.010404,0.003019,0.004022,0.8,11,5,4,5,0.7,...,0.000032,80,0.158622,0.158131,0.155717,0.160664,0.155316,0.157690,0.001972,55
97,0.706582,0.065395,0.013193,0.003221,0.8,21,7,8,17,0.2,...,0.000085,90,0.143347,0.145216,0.142723,0.149259,0.145004,0.145110,0.002282,97
98,0.664561,0.039971,0.013079,0.003191,0.6,21,10,4,18,0.4,...,0.000142,56,0.155138,0.153406,0.153489,0.159773,0.154694,0.155300,0.002335,61


In [ ]:
# export to excel
pd.DataFrame(rs.cv_results_).to_excel('GBR_RS_unscal.xlsx', index=False)

BAYESSIAN

In [ ]:
def gb_regression_cv(learning_rate,n_estimators, subsample, min_samples_split, min_samples_leaf, max_depth):
    gbr = GradientBoostingRegressor(
        learning_rate=learning_rate,
        n_estimators=int(n_estimators),
        subsample=(subsample),
        min_samples_split=int(min_samples_split),
        min_samples_leaf=int(min_samples_leaf),
        max_depth=int(max_depth)
    )
    scores = cross_val_score(gbr, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    rmse = np.sqrt(-scores.mean())
    return -rmse  # Return only the RMSE score

In [ ]:
bayes_opt = BayesianOptimization(
    gb_regression_cv,
    {
        'learning_rate': (0.01, 1.0),
        'n_estimators': (1, 51),
        'subsample': (0.5, 1.0),
        'min_samples_split': (2, 11),
        'min_samples_leaf': (1, 11),
        'max_depth': (1, 26),
    }
)

In [ ]:
start_time = time.time()
bayes_opt.maximize(init_points=10, n_iter=100)
print("Execution time: " + str((time.time() - start_time)) + ' s')

|   iter    |  target   | learni... | max_depth | min_sa... | min_sa... | n_esti... | subsample |
-------------------------------------------------------------------------------------------------
| 1         | -0.008279 | 0.7414    | 22.12     | 3.164     | 10.89     | 36.31     | 0.938     |
| 2         | -0.007243 | 0.9833    | 6.068     | 6.641     | 7.931     | 38.89     | 0.923     |
| 3         | -0.009338 | 0.8205    | 1.174     | 8.327     | 2.545     | 42.44     | 0.7068    |
| 4         | -0.009037 | 0.8082    | 25.27     | 4.487     | 10.71     | 29.53     | 0.615     |
| 5         | -0.007716 | 0.6638    | 10.36     | 3.981     | 4.39      | 42.16     | 0.8137    |
| 6         | -0.007668 | 0.4051    | 19.47     | 2.957     | 5.602     | 13.23     | 0.7488    |
| 7         | -0.008407 | 0.4082    | 19.07     | 1.142     | 2.374     | 31.55     | 0.588     |
| 8         | -0.007269 | 0.8246    | 2.661     | 9.351     | 9.693     | 49.76     | 0.7843    |
| 9         | -0.007

In [ ]:
best_iter = np.argmin([res['target'] for res in bayes_opt.res])

In [ ]:
best_params = bayes_opt.res[best_iter]['params']

print("Best Parameters:", best_params)

Best Parameters: {'learning_rate': 0.01, 'max_depth': 7.245537141767499, 'min_samples_leaf': 7.393043117365606, 'min_samples_split': 7.645624547510046, 'n_estimators': 37.461717431558576, 'subsample': 1.0}


In [ ]:
best_bs = GradientBoostingRegressor(
    learning_rate=(best_params['learning_rate']),
    n_estimators=int(best_params['n_estimators']),
    subsample=(best_params['subsample']),
    min_samples_split=int(best_params['min_samples_split']),
    min_samples_leaf=int(best_params['n_estimators']),
    max_depth=int(best_params['max_depth']),
    loss='squared_error'
)
best_bs.fit(X_train, y_train)

GradientBoostingRegressor(learning_rate=0.01, max_depth=7, min_samples_leaf=37,
                          min_samples_split=7, n_estimators=37)

In [ ]:
y_pred_bs = best_bs.predict(X_test)
rmse_bs = np.sqrt(mean_squared_error(y_test, y_pred_bs))
mape_bs = mean_absolute_percentage_error(y_test, y_pred_bs)

print(f"RMSE: {rmse_bs:.2f}")
print(f"MAPE: {mape_bs:.2f}%")

RMSE: 0.01
MAPE: 0.62%
